In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf
from tensorflow.contrib import layers
import numpy as np

env = gym.make("FrozenLake-v0")


hdf5 not supported (please install/reinstall h5py)


[2017-01-25 19:02:11,017] Making new env: FrozenLake-v0


In [2]:
n_actions = env.action_space.n
n_states = env.observation_space.n
learning_rate = 0.85
y = 0.9
model_name = "deep-integrated-policy-gradient.model"
model_path = "/models/" + model_name

graph = tf.Graph()
gsess = tf.InteractiveSession(graph=graph)
with graph.as_default():
    with tf.device("cpu:0"):
        s = tf.placeholder(tf.int32, ())
        step = tf.placeholder(tf.int32, ())
        a = tf.placeholder(tf.int32, ())
        r = tf.placeholder(tf.float32, ())
        

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        [Ps, V] = Pipe(
            s,
            T.one_hot(n_states).expand_dims(0)
            .elu_layer(32, scope='relu_layer', variables_collections=["actor"], **ops),
            [
                T.softmax_layer(n_actions, scope='softmax_layer_actor', **ops)
                >> T[0]
            ,
                T.linear_layer(1, scope='linear_layer_critic', **ops)
                >> T[0,0]
            ]
        )
        Psa = Ps[a]
        
        ws = tf.trainable_variables()
        
        gradients_actor = tf.gradients(tf.log(Psa), ws)
        gradients_actor = [ g if g is not None else tf.zeros_like(w) for g, w in zip(gradients_actor, ws) ]
        
        gradients_critic = tf.gradients(V, ws)
        gradients_critic = [ g if g is not None else tf.zeros_like(w) for g, w in zip(gradients_critic, ws) ]
        
        dws = [ tf.placeholder(w.dtype, w.get_shape()) for w in ws ]
        update = [ tf.assign_add(w, ws) for w, ws in zip(ws, dws) ]
        

        writer = tf.summary.FileWriter('/logs/' +  model_name)
        saver = tf.train.Saver()

In [3]:
def next_action(actions, get_max=False):
    n = actions.shape[0]
    return choice(n, p=actions) if not get_max else np.argmax(actions)
            
            
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, model_path)
    
    r_total = 0
    
    for t in range(200000):
        k = 20000.0
        lr = 0.5 * k / (k + t)
#         lr = 0.2
        
        _s = env.reset()
        I = 1.0
        
        while True:
            #calculate Ps
            [_ps, _v]= sess.run([Ps, V], feed_dict={s: _s})
            
            #select action
            _a = next_action(_ps)
            
            #take step
            _s1, _r, done, info = env.step(_a)
            r_total += _r
            
            #
            _v1 = sess.run(V, feed_dict={s: _s1})
            
            #train
            [_gradients_actor, _gradients_critic] = sess.run([gradients_actor, gradients_critic], {s: _s, a: _a})
            
            d = _r + y * _v1 - _v
            feed_dict = { 
                dw: lr * d * (dp_dw + dv_dw) for dw, dp_dw, dv_dw in zip(dws, _gradients_actor, _gradients_critic)
            }
            
            sess.run(update, feed_dict=feed_dict)
            
            #update state
            _s = _s1
#             I *= y

            if done:
                break
            
        

        if t % 500 == 0:
            print(int(r_total), "of", 500)
#             print(t_ward)
            r_total = 0
            saver.save(sess, model_path)


(0, 'of', 500)
(8, 'of', 500)
(7, 'of', 500)
(14, 'of', 500)
(15, 'of', 500)
(18, 'of', 500)
(14, 'of', 500)
(16, 'of', 500)
(25, 'of', 500)
(40, 'of', 500)
(107, 'of', 500)
(66, 'of', 500)
(63, 'of', 500)
(122, 'of', 500)
(174, 'of', 500)
(150, 'of', 500)
(96, 'of', 500)
(94, 'of', 500)
(77, 'of', 500)
(144, 'of', 500)
(148, 'of', 500)
(128, 'of', 500)
(73, 'of', 500)
(40, 'of', 500)
(51, 'of', 500)
(38, 'of', 500)
(35, 'of', 500)
(15, 'of', 500)
(23, 'of', 500)
(31, 'of', 500)
(40, 'of', 500)
(30, 'of', 500)
(33, 'of', 500)
(60, 'of', 500)
(53, 'of', 500)
(76, 'of', 500)
(55, 'of', 500)
(85, 'of', 500)
(94, 'of', 500)
(83, 'of', 500)
(75, 'of', 500)
(66, 'of', 500)
(66, 'of', 500)
(82, 'of', 500)
(63, 'of', 500)
(80, 'of', 500)
(81, 'of', 500)
(92, 'of', 500)
(66, 'of', 500)
(194, 'of', 500)
(179, 'of', 500)
(141, 'of', 500)
(128, 'of', 500)
(246, 'of', 500)
(332, 'of', 500)
(330, 'of', 500)
(342, 'of', 500)
(278, 'of', 500)
(348, 'of', 500)
(369, 'of', 500)
(387, 'of', 500)
(304, 'o

KeyboardInterrupt: 

In [5]:
_s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    saver.restore(sess, model_path)
    
    for i in range(1000):
        
        _ps = sess.run(Ps, feed_dict={s: _s})
        _a = next_action(_ps, 0)
        
        
        print _s
        
        print [_psa for _psa in _ps], _a
        
        _s, _r, done, info = env.step(_a)
        env.render()
        
        print _s
        
        print("")

        if done:
            print(_r)
            break

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
4

4
[0.99978358, 0.00011137999, 5.8657319e-05, 4.6419729e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
4

4
[0.99978358, 0.00011137999, 5.8657319e-05, 4.6419729e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FHFH
FFFH
HFFG
  (Left)
0

0
[0.99826837, 0.0013400769, 0.00032935015, 6.2188512e-05] 0
SFFF
FH